In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import os.path


file_path ='/home/akurdi/Desktop/SDP/datasetcsv'

print ('is folder found')
print (os.path.exists(file_path))

orinigal_dataset_path=[]
for dirname, _, filenames in os.walk(file_path):
    print(dirname)
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if filename.endswith ('csv') :
            orinigal_dataset_path.append(os.path.join(dirname, filename))
        

In [ ]:
 import pandas as pd
print('number of dataset' , len (orinigal_dataset_path))

dataSets =[]

for dataset in orinigal_dataset_path:
    print(dataset)
    orig_df = pd.read_csv(dataset)
    print(orig_df.shape)
    # orig_df.head(3)
   
    orig_df.replace( '?', 0, inplace=True)
    # Assuming 'orig_df' is your dataframe and 'defects' contains True/False values
    # orig_df['class'] = orig_df['class'].astype(int)
    if 'class' in orig_df.columns:
        orig_df['class'] = orig_df['class'].replace({'clean': 0, 'buggy': 1}).astype(int)
        X = orig_df.drop('class', axis=1)  # Features (drop the target column)
        y = orig_df['class']               # Target column
        orig_df.rename(columns={'class': 'defects'}, inplace=True)
    elif 'defects' in orig_df.columns:
        orig_df['defects'] = orig_df['defects'].astype(int)
        X = orig_df.drop('defects', axis=1)  # Features (drop the target column)
        y = orig_df['defects']               # Target column
    elif 'bug' in orig_df.columns:
        X = orig_df.drop('bug', axis=1)  # Features (drop the target column)
        y = orig_df['bug']
        orig_df.rename(columns={'bug': 'defects'}, inplace=True)
    elif 'isDefective' in orig_df.columns:
        orig_df['isDefective'] = orig_df['isDefective'].replace({'clean': 0, 'buggy': 1}).astype(int)
        X = orig_df.drop('isDefective', axis=1)  # Features (drop the target column)
        y = orig_df['isDefective']
        orig_df.rename(columns={'isDefective': 'defects'}, inplace=True)
    elif 'Defective' in orig_df.columns:
        orig_df['Defective'] = orig_df['Defective'].replace({'N': 0, 'Y': 1}).astype(int)
        X = orig_df.drop('Defective', axis=1)  # Features (drop the target column)
        y = orig_df['Defective']
        orig_df.rename(columns={'Defective': 'defects'}, inplace=True)
    elif 'c' in orig_df.columns:
        orig_df['c'] = orig_df['c'].astype(int)
        X = orig_df.drop('c', axis=1)  # Features (drop the target column)
        y = orig_df['c']    
        orig_df.rename(columns={'c': 'defects'}, inplace=True)

    # Rename the target column y to defects
    y = orig_df['defects']
    dataSets.append(orig_df)
    print(orig_df[X.columns].apply(pd.to_numeric, errors='coerce').isnull().sum())

    # Assuming 'orig_df' is your dataset and it has features and a target column
  

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

model = ExtraTreesClassifier()
for dataset in dataSets:
    # print(dataset)
    X = orig_df.drop('defects', axis=1)  
    y = orig_df['defects']
    model.fit(X,y)
    print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(35).plot(kind='barh')
    plt.show()


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
         # Target column


# Initialize KFold with 20 splits
kf = KFold(n_splits=2, shuffle=True, random_state=5)


# Lists to store the metrics for each fold


# Define different gamma and C values for the RBF kernel (4 combinations)
gamma_values = [0.01, 0.1, 1, 10]  # You can adjust these gamma values
C_values = [0.1, 1, 10, 100]       # You can adjust these C values

# Loop through the different kernel variations (RBF with different gamma and C)

accuracies = []
precisions = []
recalls = []
f1_scores = []

for dataset in dataSets:
    
    # Assuming 'orig_df' is your dataset and it has features and a target column
    X = dataset.drop('defects', axis=1)  # Features (drop the target column)
    # print(dataset[X.columns].apply(pd.to_numeric, errors='coerce').isnull().sum())
    y = dataset['defects']

    # To store results of each fold (optional)
    for gamma, C in zip(gamma_values, C_values):
        fold_num = 1

        for train_index, test_index in kf.split(X):
        # Split data into train and test for this fold
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
            # Initialize the SVM model with RBF kernel and the current gamma, C values
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
        
            # Train the model on the training data
            clf.fit(X_train, y_train)

            # Predict on the test data
            y_pred = clf.predict(X_test)

            # Calculate metrics
            DT_accuracy = accuracy_score(y_test, y_pred)
            DT_precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' accounts for imbalanced classes
            DT_recall = recall_score(y_test, y_pred, average='weighted')
            DT_f1 = f1_score(y_test, y_pred, average='weighted')

            # Append metrics to lists
            accuracies.append(DT_accuracy)
            precisions.append(DT_precision)
            recalls.append(DT_recall)
            f1_scores.append(DT_f1)

            # Print metrics for this kernel variation
            print("Accuracy: {:.2f}".format(DT_accuracy))
            print("Precision: {:.2f}".format(DT_precision))
            print("Recall: {:.2f}".format(DT_recall))
            print("F1 score: {:.2f}".format(DT_f1))
            print("-" * 30)
            fold_num += 1

        print(f"Fold {fold_num}, RBF Kernel with gamma={gamma} and C={C}:")
        # After all folds, calculate the mean and standard deviation of each metric
        mean_accuracy = np.mean(accuracies)
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        mean_f1 = np.mean(f1_scores)

        std_accuracy = np.std(accuracies)
        std_precision = np.std(precisions)
        std_recall = np.std(recalls)
        std_f1 = np.std(f1_scores)

        # Print the compiled results
        print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
        print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
        print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
        print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))

# After all folds, calculate the mean and standard deviation of each metric
mean_accuracy = np.mean(accuracies)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)
mean_f1 = np.mean(f1_scores)

std_accuracy = np.std(accuracies)
std_precision = np.std(precisions)
std_recall = np.std(recalls)
std_f1 = np.std(f1_scores)

# Print the compiled results
print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

# Initialize KFold with 20 splits
kf = KFold(n_splits=10, shuffle=True, random_state=5)


# Lists to store the metrics for each fold


# Define different gamma and C values for the RBF kernel (4 combinations)
gamma_values = [0.01, 0.1, 1, 10]  # You can adjust these gamma values
C_values = [0.1, 1, 10, 100]       # You can adjust these C values

# Loop through the different kernel variations (RBF with different gamma and C)

accuracies = []
precisions = []
recalls = []
f1_scores = []

for dataset in dataSets:
    
    # Assuming 'orig_df' is your dataset and it has features and a target column
    X = dataset.drop('defects', axis=1)  # Features (drop the target column)
    # print(dataset[X.columns].apply(pd.to_numeric, errors='coerce').isnull().sum())
    y = dataset['defects']

    # To store results of each fold (optional)
    for gamma, C in zip(gamma_values, C_values):
        fold_num = 1

        for train_index, test_index in kf.split(X):
            # Split data into train and test for this fold
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
            # Initialize ExtraTreesClassifier for feature selection
            model = ExtraTreesClassifier(random_state=42)
            model.fit(X_train, y_train)

            num_features_to_select = int(0.7 * X_train.shape[1])
            feature_importances = model.feature_importances_
            indices = np.argsort(feature_importances)[::-1]  # Get indices of the sorted feature importances in descending order
            selected_features = X_train.columns[indices[:num_features_to_select] ]  # Select the corresponding feature names
            print(f"Top ",num_features_to_select, selected features: {list(selected_features)}")

            # Transform the train and test sets to only include the selected features
            X_train_selected = X_train[selected_features]
            X_test_selected = X_test[selected_features]

            # Plot graph of feature importances only for selected features
            selected_feat_importances = pd.Series(feature_importances, index=X_train.columns)[selected_features]
            selected_feat_importances.sort_values(ascending=False).plot(kind='barh')
            plt.title("Feature Importances for Selected Features top")
            plt.show()

            # Initialize the SVM model
            clf = SVC(kernel='rbf', C=C, gamma=gamma)

            # Train the SVM model on the selected features
            clf.fit(X_train_selected, y_train)
    
            # Predict on the test data with selected features
            y_pred = clf.predict(X_test_selected)
    
            # Split data into train and test for this fold
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
            # Calculate metrics
            DT_accuracy = accuracy_score(y_test, y_pred)
            DT_precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' accounts for imbalanced classes
            DT_recall = recall_score(y_test, y_pred, average='weighted')
            DT_f1 = f1_score(y_test, y_pred, average='weighted')

            # Append metrics to lists
            accuracies.append(DT_accuracy)
            precisions.append(DT_precision)
            recalls.append(DT_recall)
            f1_scores.append(DT_f1)

            # Print metrics for this kernel variation
            print("Accuracy: {:.2f}".format(DT_accuracy))
            print("Precision: {:.2f}".format(DT_precision))
            print("Recall: {:.2f}".format(DT_recall))
            print("F1 score: {:.2f}".format(DT_f1))
            print("-" * 30)
            fold_num += 1

        print(f"Fold {fold_num}, RBF Kernel with gamma={gamma} and C={C}:")
        # After all folds, calculate the mean and standard deviation of each metric
        mean_accuracy = np.mean(accuracies)
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        mean_f1 = np.mean(f1_scores)

        std_accuracy = np.std(accuracies)
        std_precision = np.std(precisions)
        std_recall = np.std(recalls)
        std_f1 = np.std(f1_scores)

        # Print the compiled results
        print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
        print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
        print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
        print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))

# After all folds, calculate the mean and standard deviation of each metric
mean_accuracy = np.mean(accuracies)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)
mean_f1 = np.mean(f1_scores)

std_accuracy = np.std(accuracies)
std_precision = np.std(precisions)
std_recall = np.std(recalls)
std_f1 = np.std(f1_scores)

# Print the compiled results
print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

# Initialize KFold with 20 splits
kf = KFold(n_splits=4, shuffle=True, random_state=5)


# Lists to store the metrics for each fold


# Define different gamma and C values for the RBF kernel (4 combinations)
gamma_values = [0.01, 0.1, 1, 10]  # You can adjust these gamma values
C_values = [0.1, 1, 10, 100]       # You can adjust these C values

# Loop through the different kernel variations (RBF with different gamma and C)

accuracies = []
precisions = []
recalls = []
f1_scores = []

for dataset in dataSets:
    
    # Assuming 'orig_df' is your dataset and it has features and a target column
    X = dataset.drop('defects', axis=1)  # Features (drop the target column)
    # print(dataset[X.columns].apply(pd.to_numeric, errors='coerce').isnull().sum())
    y = dataset['defects']

    # To store results of each fold (optional)
    for gamma, C in zip(gamma_values, C_values):
        fold_num = 1

        for train_index, test_index in kf.split(X):
            # Split data into train and test for this fold
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
            # Initialize ExtraTreesClassifier for feature selection
            model = RandomForestClassifier(random_state=42)
            model.fit(X_train, y_train)

            num_features_to_select = int(0.7 * X_train.shape[1])
            feature_importances = model.feature_importances_
            indices = np.argsort(feature_importances)[::-1]  # Get indices of the sorted feature importances in descending order
            selected_features = X_train.columns[indices[:num_features_to_select] ]  # Select the corresponding feature names
                    # Calculate number of features to select as 60% of total features

            print(f"Top ",num_features_to_select," selected features: {list(selected_features)}")

            # Transform the train and test sets to only include the selected features
            X_train_selected = X_train[selected_features]
            X_test_selected = X_test[selected_features]

            # Plot graph of feature importances only for selected features
            selected_feat_importances = pd.Series(feature_importances, index=X_train.columns)[selected_features]
            selected_feat_importances.sort_values(ascending=False).plot(kind='barh')
            plt.title("Feature Importances for Selected Features top")
            plt.show()

            # Initialize the SVM model
            clf = SVC(kernel='rbf', C=C, gamma=gamma)

            # Train the SVM model on the selected features
            clf.fit(X_train_selected, y_train)
    
            # Predict on the test data with selected features
            y_pred = clf.predict(X_test_selected)
    
            # Split data into train and test for this fold
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
            # Calculate metrics
            DT_accuracy = accuracy_score(y_test, y_pred)
            DT_precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' accounts for imbalanced classes
            DT_recall = recall_score(y_test, y_pred, average='weighted')
            DT_f1 = f1_score(y_test, y_pred, average='weighted')

            # Append metrics to lists
            accuracies.append(DT_accuracy)
            precisions.append(DT_precision)
            recalls.append(DT_recall)
            f1_scores.append(DT_f1)

            # Print metrics for this kernel variation
            print("Accuracy: {:.2f}".format(DT_accuracy))
            print("Precision: {:.2f}".format(DT_precision))
            print("Recall: {:.2f}".format(DT_recall))
            print("F1 score: {:.2f}".format(DT_f1))
            print("-" * 30)
            fold_num += 1

        print(f"Fold {fold_num}, RBF Kernel with gamma={gamma} and C={C}:")
        # After all folds, calculate the mean and standard deviation of each metric
        mean_accuracy = np.mean(accuracies)
        mean_precision = np.mean(precisions)
        mean_recall = np.mean(recalls)
        mean_f1 = np.mean(f1_scores)

        std_accuracy = np.std(accuracies)
        std_precision = np.std(precisions)
        std_recall = np.std(recalls)
        std_f1 = np.std(f1_scores)

        # Print the compiled results
        print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
        print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
        print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
        print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))

# After all folds, calculate the mean and standard deviation of each metric
mean_accuracy = np.mean(accuracies)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)
mean_f1 = np.mean(f1_scores)

std_accuracy = np.std(accuracies)
std_precision = np.std(precisions)
std_recall = np.std(recalls)
std_f1 = np.std(f1_scores)

# Print the compiled results
print("Mean Accuracy: {:.2f} ± {:.2f}".format(mean_accuracy, std_accuracy))
print("Mean Precision: {:.2f} ± {:.2f}".format(mean_precision, std_precision))
print("Mean Recall: {:.2f} ± {:.2f}".format(mean_recall, std_recall))
print("Mean F1 Score: {:.2f} ± {:.2f}".format(mean_f1, std_f1))
